# Filber 源码

## ReactUpdateQueue.js

UpdateQueue is a linked list of prioritized updates.

like fibers, update queues come in pairs: a current queue, which represents the visible state of the screen,
and a work-in-progress queue, which can be mutated and processed asynchoronously before it is committed - a form of double buffering.
if a work-in-progress render is discarded befor finishing, we create a new work-in-progress by cloning the current queue.

both queues share a persistent, singly-linked list structure. to schedule an update, we append it to the end of both queues.
each queue maintains a pointer to first update in the persistent list that hasn't been processed.
the work-in-progress pointer always has a position equal to or greater than the current queue,
since we always work on that one. the current queue's pointer is only updated during the commit phase, 
when we swap in the work-in-progree.

the reson we append to both queues is bacause otherwise we might drop updates without ever processing them. 

**Prioritization**

updates are not sorted by priority, but by insertion; new updates are always appended to the end of the list.

the priority is still important , thought. when processing the update queue during the render phase, only the updates 
with sufficient priority are included in the result. if we skip an update because it has insufficient priority , 
it remains in the queue to be processed later, during a lower priority render. crucially , all updates subsequent to a skipped update also
remain in the queue *regardless of their priority*. that means high priority updates are sometimes processed twice , at two separate priorities.
we also keep track of a base state, that represents the state before the first update in the queue is applied.

```javascript
export function processUpdateQueue<State>(
  workInProgress: Fiber,
  queue: UpdateQueue<State>,
  props: any,
  instance: any,
  renderExpirationTime: ExpirationTime,
): void {
  hasForceUpdate = false;

  queue = ensureWorkInProgressQueueIsAClone(workInProgress, queue); 

  // These values may change as we process the queue.
  let newBaseState = queue.baseState;
  let newFirstUpdate = null;
  let newExpirationTime = NoWork;

  // Iterate through the list of updates to compute the result.
  let update = queue.firstUpdate;
  let resultState = newBaseState;
  while (update !== null) {
    const updateExpirationTime = update.expirationTime;
    if (updateExpirationTime < renderExpirationTime) {
      // This update does not have sufficient priority. Skip it.
      if (newFirstUpdate === null) {
        // This is the first skipped update. It will be the first update in
        // the new list.
        newFirstUpdate = update;
        // Since this is the first update that was skipped, the current result
        // is the new base state.
        newBaseState = resultState;
      }
      // Since this update will remain in the list, update the remaining
      // expiration time.
      if (newExpirationTime < updateExpirationTime) {
        newExpirationTime = updateExpirationTime;
      }
    } else {
      // This update does have sufficient priority. Process it and compute
      // a new result.
      resultState = getStateFromUpdate(
        workInProgress,
        queue,
        update,
        resultState,
        props,
        instance,
      );
      const callback = update.callback;
      if (callback !== null) {
        workInProgress.effectTag |= Callback;
        // Set this to null, in case it was mutated during an aborted render.
        update.nextEffect = null;
        if (queue.lastEffect === null) {
          queue.firstEffect = queue.lastEffect = update;
        } else {
          queue.lastEffect.nextEffect = update;
          queue.lastEffect = update;
        }
      }
    }
    // Continue to the next update.
    update = update.next;
  }

  // Separately, iterate though the list of captured updates.
  let newFirstCapturedUpdate = null;
  update = queue.firstCapturedUpdate;
  while (update !== null) {
    const updateExpirationTime = update.expirationTime;
    if (updateExpirationTime < renderExpirationTime) {
      // This update does not have sufficient priority. Skip it.
      if (newFirstCapturedUpdate === null) {
        // This is the first skipped captured update. It will be the first
        // update in the new list.
        newFirstCapturedUpdate = update;
        // If this is the first update that was skipped, the current result is
        // the new base state.
        if (newFirstUpdate === null) {
          newBaseState = resultState;
        }
      }
      // Since this update will remain in the list, update the remaining
      // expiration time.
      if (newExpirationTime < updateExpirationTime) {
        newExpirationTime = updateExpirationTime;
      }
    } else {
      // This update does have sufficient priority. Process it and compute
      // a new result.
      resultState = getStateFromUpdate(
        workInProgress,
        queue,
        update,
        resultState,
        props,
        instance,
      );
      const callback = update.callback;
      if (callback !== null) {
        workInProgress.effectTag |= Callback;
        // Set this to null, in case it was mutated during an aborted render.
        update.nextEffect = null;
        if (queue.lastCapturedEffect === null) {
          queue.firstCapturedEffect = queue.lastCapturedEffect = update;
        } else {
          queue.lastCapturedEffect.nextEffect = update;
          queue.lastCapturedEffect = update;
        }
      }
    }
    update = update.next;
  }

  if (newFirstUpdate === null) {
    queue.lastUpdate = null;
  }
  if (newFirstCapturedUpdate === null) {
    queue.lastCapturedUpdate = null;
  } else {
    workInProgress.effectTag |= Callback;
  }
  if (newFirstUpdate === null && newFirstCapturedUpdate === null) {
    // We processed every update, without skipping. That means the new base
    // state is the same as the result state.
    newBaseState = resultState;
  }

  queue.baseState = newBaseState;
  queue.firstUpdate = newFirstUpdate;
  queue.firstCapturedUpdate = newFirstCapturedUpdate;

  // Set the remaining expiration time to be whatever is remaining in the queue.
  // This should be fine because the only two other things that contribute to
  // expiration time are props and context. We're already in the middle of the
  // begin phase by the time we start processing the queue, so we've already
  // dealt with the props. Context in components that specify
  // shouldComponentUpdate is tricky; but we'll have to account for
  // that regardless.
  workInProgress.expirationTime = newExpirationTime;
  workInProgress.memoizedState = resultState;
}
```

## ReactFiberReconciler.js

```javascript
    function scheduleRootUpdate(
      current: Fiber,
      element: ReactNodeList,
      expirationTime: ExpirationTime,
      callback: ?Function,
    ) {   
          
      const update = createUpdate(expirationTime);
      // Caution: React DevTools currently depends on this property
      // being called "element".
      update.payload = {element};

      callback = callback === undefined ? null : callback;
      if (callback !== null) {
        warningWithoutStack(
          typeof callback === 'function',
          'render(...): Expected the last optional `callback` argument to be a ' +
            'function. Instead received: %s.',
          callback,
        );
        update.callback = callback;
      }

      flushPassiveEffects();
      enqueueUpdate(current, update);
      scheduleWork(current, expirationTime);

      return expirationTime;
    }
```



## ReactFiberHooks.js

hooks:
1. useState
1. useReducer
1. useContext
1. useRef
1. useEffect
1. useLayoutEffect
1. useCallback
1. useMemo
1. useImperativeHandle
1. useDebugValue

In [5]:
{
interface FiberType {
    effect:boolean,
    sibling:FiberType|null,
    next:FiberType|null,
    return:FiberType|null,
    value:any
}
interface NodeType {
    label:string,
    attr:string[],
    text:string
}

class Fiber implements FiberType {
    node:NodeType|nul;l;
    effect:boolean;
    sibling:FiberType|null;
    next:FiberType|null;
    return:FiberType|null;
    constructor(effect:boolean,node:NodeType|null,sibling:FiberType|null,next:FiberType|null){
        this.effect = effect;
        this.sibling = sibling;
        this.next = next;
        this.node = node;
    }
    
}

let a = new Fiber(false,1,null,null);
console.log(a);
}

Fiber { effect: false, siblings: null, next: null, value: 1 }


undefined

In [ ]:
{
    var htmlexp = /("[^"]*"|'[^']*')/;
    function parse(html:string):string {
        
    }
}